# Image Classification using VGG16_BN 

In [1]:
# install dependencies and packages
!pip install torch torchvision scikit-learn cnn_finetune


In [2]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

# cnn_finetune imports caused problems, using torchvision directly instead
from torchvision import transforms, models
from sklearn.model_selection import train_test_split
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

#os.environ["TORCHDYNAMO_DISABLE"] = "1"

A custom Dataset class is created to load the coral images from the file paths HEALTHY_IMAGES_DIR and BLEACHED_IMAGES_DIR.
The subfolders contain images of healthy and bleached corals.

The dataset used is from Kaggle: https://www.kaggle.com/datasets/vencerlanz09/healthy-and-bleached-corals-image-classification/data

Dataset Details:
+ Total images: 923
+ Image categories: 2
    + Healthy corals: 438 images
    + Bleached corals: 485 images
+ Image format: JPEG
+ Image size: Maximum 300 px for either width or height, whichever is higher


In [ ]:
# prepare coral dataset

class CoralDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert('RGB')
        if self.transform:
            image = self.transform(image)
        label = self.labels[idx]
        return image, label
    
    
HEALTHY_IMAGES_DIR = "/data/healthy_corals"
BLEACHED_IMAGES_DIR = "/data/bleached_corals"

# imagepaths and labels
healthy_image_paths = [os.path.join(HEALTHY_IMAGES_DIR, img) for img in os.listdir(HEALTHY_IMAGES_DIR) if os.path.isfile(os.path.join(HEALTHY_IMAGES_DIR, img))]
bleached_image_paths = [os.path.join(BLEACHED_IMAGES_DIR, img) for img in os.listdir(BLEACHED_IMAGES_DIR) if os.path.isfile(os.path.join(BLEACHED_IMAGES_DIR, img))]

image_paths = healthy_image_paths + bleached_image_paths
labels = [0] * len(healthy_image_paths) + [1] * len(bleached_image_paths)

FileNotFoundError: [Errno 2] No such file or directory: './data/healthy_corals'

The datasets is splitted into training (80%), validation (10%) and testing (10%).
Training data is used for model learning, the validation data evaluates the performance during training and the data assesses the final model performance.

In [ ]:
# split dataset into training (80%), validation (10%) and testing (10%)
train_paths, temp_paths, train_labels, temp_labels = train_test_split(image_paths, labels, test_size=0.2, random_state=42)
val_paths, test_paths, val_labels, test_labels = train_test_split(temp_paths, temp_labels, test_size=0.5, random_state=42)

print('Dataset was successfully split into training, validation, and testing sets')

Dataset was successfully split into training, validation, and testing sets


For the Loss Function ```nn.CrossEntropyLoss``` because it is suitable for multi-class classification problems 

For the Optimizer Adam is chosen. Other optimizers like SGD can be used aswell to evaluate the performance.

A scheduler is used to adjust the learning rate dynamically during training to optimize the learning process.

In [ ]:
# Define runs with different parameters
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

runs = [
    {"batch_size": 32, "lr": 0.001, "augmentation": "default", "epochs": 30, "step_size": 5},
    {"batch_size": 64, "lr": 0.0001, "augmentation": "horizontal_flip", "epochs": 50, "step_size": 10},
    {"batch_size": 16, "lr": 0.0005, "augmentation": "rotation", "epochs": 30, "step_size": 5},
    {"batch_size": 48, "lr": 0.0002, "augmentation": "color_jitter", "epochs": 40, "step_size": 8},
]

# Define augmentation options
def get_transforms(augmentation_type):
    if augmentation_type == "default":
        return transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=mean, std=std),
        ])
    elif augmentation_type == "horizontal_flip":
        return transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(mean=mean, std=std),
        ])
    elif augmentation_type == "rotation":
        return transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.RandomRotation(15),
            transforms.ToTensor(),
            transforms.Normalize(mean=mean, std=std),
        ])
    elif augmentation_type == "color_jitter":
        return transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.RandomHorizontalFlip(),
            transforms.ColorJitter(brightness=0.2, contrast=0.2),
            transforms.ToTensor(),
            transforms.Normalize(mean=mean, std=std),
        ])

# Training loop for dynamic runs
results = []

# create model and adapt it to coral Dataset
model = models.vgg16_bn(pretrained=True)
model.classifier[6] = nn.Linear(model.classifier[6].in_features, 2)

# move model to the gpu if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


for i, run in enumerate(runs):
    print(f"\n### Running Configuration {i + 1} ###\n")
    
    # Set up the dataset and dataloaders
    transform = get_transforms(run["augmentation"])
    train_dataset = CoralDataset(train_paths, train_labels, transform=transform)
    val_dataset = CoralDataset(val_paths, val_labels, transform=transform)
    train_loader = DataLoader(train_dataset, batch_size=run["batch_size"], shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=run["batch_size"], shuffle=False)

    # Define optimizer and loss function
    optimizer = optim.Adam(model.parameters(), lr=run["lr"])
    criterion = nn.CrossEntropyLoss()
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=run["step_size"], gamma=0.1)

    # Training loop
    for epoch in range(run["epochs"]):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        # Validation loop
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_accuracy = 100 * correct / total
        print(f"Epoch [{epoch+1}/{run['epochs']}], Loss: {running_loss/len(train_loader):.4f}, Val Loss: {val_loss/len(val_loader):.4f}, Val Accuracy: {val_accuracy:.2f}%")

    # Log results for this run
    results.append({
        "run": i + 1,
        "batch_size": run["batch_size"],
        "lr": run["lr"],
        "augmentation": run["augmentation"],
        "epochs": run["epochs"],
        "final_val_loss": val_loss / len(val_loader),
        "final_val_accuracy": val_accuracy,
    })

# Display results
results_df = pd.DataFrame(results)
print("\nFinal Results:")
print(results_df)


### Running Configuration 1 ###

Epoch [1/30], Loss: 0.9006, Val Loss: 10.3522, Val Accuracy: 67.39%
Epoch [2/30], Loss: 0.6492, Val Loss: 4.1800, Val Accuracy: 70.65%
Epoch [3/30], Loss: 0.6614, Val Loss: 3.2572, Val Accuracy: 53.26%
Epoch [4/30], Loss: 0.6715, Val Loss: 1.2374, Val Accuracy: 72.83%
Epoch [5/30], Loss: 0.5579, Val Loss: 0.4821, Val Accuracy: 73.91%
Epoch [6/30], Loss: 0.5174, Val Loss: 0.5648, Val Accuracy: 69.57%
Epoch [7/30], Loss: 0.5098, Val Loss: 0.8162, Val Accuracy: 67.39%
Epoch [8/30], Loss: 0.4903, Val Loss: 0.5649, Val Accuracy: 71.74%
Epoch [9/30], Loss: 0.4947, Val Loss: 0.9814, Val Accuracy: 73.91%


KeyboardInterrupt: 